In [9]:
import pandas as pd
import tsfresh

# Correlation Code

## training

In [3]:
# read in trait data
train_trait = pd.read_csv('../Maize_GxE_Competition_Data/Training_Data/1_Training_Trait_Data_2014_2021.csv')
# read in EC data
ec_data =  pd.read_csv('../Maize_GxE_Competition_Data/Training_Data/6_Training_EC_Data_2014_2021.csv')

# set environment as rownames
ec_data.index = ec_data['Env']
ec_data = ec_data.drop('Env', axis=1)

# calculate pairwise correlation between features
ec_data_corr = ec_data.corr().abs()

# might need to: pip install tsfresh
from tsfresh.feature_selection.relevance import calculate_relevance_table

# feature association test with avg yield per environment
y = train_trait.loc[train_trait['Env'].isin(ec_data.index),:].groupby('Env')['Yield_Mg_ha'].mean()
features_relevance_table = calculate_relevance_table(ec_data, y) # order by feature significance (p-val)

# filter highly correlated features
corr_thresh = 0.8
i = 0
filtered_features_relevance_table = features_relevance_table
while i < filtered_features_relevance_table.shape[0]:
    feature = filtered_features_relevance_table['feature'][i]
    corr_features = ec_data_corr.index[ec_data_corr.loc[feature] > corr_thresh].drop(feature, errors='ignore')
    corr_features = corr_features[corr_features.isin(filtered_features_relevance_table.index)]
    filtered_features_relevance_table = filtered_features_relevance_table.drop(corr_features, axis=0)
    i = i + 1
    
filtered_features = filtered_features_relevance_table.index

filtered_features_relevance_table.to_csv("EC_feature_relevance.csv")

# Testing

# imputation

## training

In [21]:
from sklearn.impute import KNNImputer

In [28]:
data = pd.read_csv('normalized_soil.csv')
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit_transform(data)
pd.DataFrame(imputer.transform(data), index=data.index, columns=data.columns).to_csv("iputed_soil_data.csv")

## testing

In [27]:
data = pd.read_csv('normalized_test_soil.csv')
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit_transform(data)
pd.DataFrame(imputer.transform(data), index=data.index, columns=data.columns).to_csv("iputed_test_soil_data.csv")